# Calorimetry likelihood stopping protons with arrays

In [1]:
%load_ext autoreload
%autoreload 2

import math
from itertools import product
import uproot
import awkward
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import binned_statistic

import matplotlib
matplotlib.rcParams.update({'font.size': 12})
from calo_likelihood import caloLikelihood
from general_functions import *
from loaders import *
from plotter import *
from categorisation import *
from initializers import *

In [2]:
# today = 'calorimetry'
today = 'calorimetry_trial2'

In [3]:
branches = [
    'run',
    'sub',
    'evt',
    
    'backtracked_pdg',
    'backtracked_e',
    'backtracked_purity',
    'backtracked_completeness',
    'backtracked_overlay_purity',
    'backtracked_end_process',
    
    'backtracked_start_x',
    'backtracked_start_y',
    'backtracked_start_z',
    
    'backtracked_sce_start_x',
    'backtracked_sce_start_y',
    'backtracked_sce_start_z',
    
    'nplanehits_U',
    'nplanehits_V',
    'nplanehits_Y',
    'trk_score',

    'generation',
    'trk_daughters',
    'shr_daughters',

    'trk_sce_start_x',
    'trk_sce_start_y',
    'trk_sce_start_z',

    'trk_sce_end_x',
    'trk_sce_end_y',
    'trk_sce_end_z',
    
    'trk_theta',
    'trk_phi',
     
    'trk_dir_x',
    'trk_dir_y',
    'trk_dir_z',

    'trk_len',
    'trk_distance',
    'longest',
    
    'trk_mcs_muon_mom',
    'trk_range_muon_mom',
    
    'is_hit_montecarlo_u',
    'is_hit_montecarlo_v',
    'is_hit_montecarlo_y',
    
    'dqdx_u',
    'dqdx_v',
    'dqdx_y',

    'dedx_u',
    'dedx_v',
    'dedx_y',

    'rr_u',
    'rr_v',
    'rr_y',

    'pitch_u',
    'pitch_v',
    'pitch_y',
    
    'dir_x_u',
    'dir_x_v',
    'dir_x_y',
    
    'dir_y_u',
    'dir_y_v',
    'dir_y_y',
    
    'dir_z_u',
    'dir_z_v',
    'dir_z_y',
]

In [67]:
directory = '/home/nic/Desktop/MicroBooNE/calorimetry_likelihood/v07/'
pot_beam_on = 4.054e+19
n_triggers_on = 8977290.0
n_triggers_off = 33240727.0

fraction = 0.1
pot_beam_on *= fraction

file_lists = {}

# file_lists['beam_on'] = directory+'beam_on/out_proton_skimmed.root'
file_lists['beam_on'] = directory+'beam_on/out.root'
# file_lists['beam_off'] = directory+'beam_off/out_proton_skimmed.root'
# file_lists['bnb_nu'] = directory+'bnb_nu/out_proton_skimmed.root'
# file_lists['bnb_dirt'] = directory+'bnb_dirt/out_proton_skimmed.root'

folder = "nuselection"
events_tree_name = "CalorimetryAnalyzer"
pot_tree_name = "SubRun"

In [68]:
arrays, pot = load_data_calo(file_lists, 
                        branches, 
                        folder, 
                        events_tree_name, 
                        pot_tree_name, 
#                         labels=['beam_on', 'beam_off', 'bnb_nu', 'bnb_dirt'],
                        labels=['beam_on'],
                        lazy=False,
                        fraction=fraction)

start loading beam_on
Done!


In [69]:
scale_factors = compute_scale_factors(pot, pot_beam_on, n_triggers_on, n_triggers_off)

In [70]:
scale_factors

{'beam_on': 1, 'beam_off': 0.27006900300345416}

In [71]:
for name, array in arrays.items():
    print(name)
    initialisation_for_caloriemtry_data_mc(array)
    compute_pid(array)

beam_on


/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/general_functions.py:33: RuntimeWarning: divide by zero encountered in true_divide
  return 0.3 / cos


In [9]:
my_plotter = plotter(arrays, scale_factors, pot_beam_on, branch_weights_name='weightSplineTimesTune')

# Base plots data/MC comparison 

In [ ]:
variables = [
    'trk_len',
    'trk_score',
    'trk_theta',
    'trk_phi',
    'llr_012_n'
]

binnings = {
    'trk_len': [50, 0, 600],
    'trk_score': [50, 0, 1],
    'trk_theta': [50, 0, np.pi],
    'trk_phi': [50, -np.pi, np.pi],
    'llr_012_n': [50, -1, 1],
}

xlabels = {
    'trk_len': 'Track length [cm]',
    'trk_score': r'Track score',
    'trk_theta': r'Track $\theta$',
    'trk_phi': r'Track $\phi$',
    'llr_012_n': 'Track PID',
}

log = {
    'trk_len': True,
    'trk_score': False,
    'trk_theta': False,
    'trk_phi': False,
    'llr_012_n': False,
}

In [ ]:
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/protons/base_plots/
save_folder_base = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/protons/base_plots/'

title_base = 'Trk-score > 0.5\ntrk-vtx distance < 5 cm, start-end fiducial'
for var in variables:
    out = my_plotter.plot_pot_normalised_var(variable=var,
                                   binning=binnings[var], 
                                   categories=pdg_categories, 
                                   additional_selection=None, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=xlabels[var], 
                                   title=title_base,
                                   log=log[var], 
                                   subtracted=False, 
                                   onebin=False)
    plt.savefig(save_folder_base + var + '_before_pid_cut_pot.png', dpi=300)
    plt.savefig(save_folder_base + var + '_before_pid_cut_pot.pdf')
    plt.close()
    
    
    out = my_plotter.plot_pot_normalised_var(variable=var,
                                   binning=binnings[var], 
                                   categories=pdg_categories, 
                                   additional_selection=None, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=xlabels[var], 
                                   title=title_base,
                                   log=log[var], 
                                   subtracted=True, 
                                   onebin=False,
                                   area_norm=True)
    plt.savefig(save_folder_base + var + '_before_pid_cut_area_norm.png', dpi=300)
    plt.savefig(save_folder_base + var + '_before_pid_cut_area_norm.pdf')
    plt.close()

In [ ]:
def additional_selection(array):
    return array['llr_012_n'] < -0.1

binnings['trk_len'] = [50, 0, 150]

title_base = 'Trk-score > 0.5, track PID < -0.1\ntrk-vtx distance < 5 cm, start-end fiducial'
for var in variables:
    out = my_plotter.plot_pot_normalised_var(variable=var,
                                   binning=binnings[var], 
                                   categories=pdg_categories, 
                                   additional_selection=additional_selection, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=xlabels[var], 
                                   title=title_base,
                                   log=False, 
                                   subtracted=False, 
                                   onebin=False)
    plt.savefig(save_folder_base + var + '_after_pid_cut_pot.png', dpi=300)
    plt.savefig(save_folder_base + var + '_after_pid_cut_pot.pdf')
    plt.close()
    
    
    out = my_plotter.plot_pot_normalised_var(variable=var,
                                   binning=binnings[var], 
                                   categories=pdg_categories, 
                                   additional_selection=additional_selection, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=xlabels[var], 
                                   title=title_base,
                                   log=False, 
                                   subtracted=True, 
                                   onebin=False,
                                   area_norm=True)
    plt.savefig(save_folder_base + var + '_after_pid_cut_area_norm.png', dpi=300)
    plt.savefig(save_folder_base + var + '_after_pid_cut_area_norm.pdf')
    plt.close()

# Plots at low rr, in bins of rr and pitch

In [ ]:
this_folder = 'low_rr_before_after'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/protons/$this_folder/plane0/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/protons/$this_folder/plane1/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/protons/$this_folder/plane2/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/protons/{}/'.format(this_folder)

parameters = ['rr', 'pitch']
parameters_legend_names = ['residual range', 'pitch']
parameters_bin_edges = [[0, 2, 5, 10, 20, 30, 50, 150, 300], [0.3, 0.4, 1, 30]]
dedx_vars = 'dqdx'

binning = [60, 0, 1500]
title_base = 'Trk-score > 0.5, track PID < -0.1\ntrk-vtx distance < 5 cm, start-end fiducial'

chi2_avant = {}
chi2_depois = {}

for plane_num, plane in zip([2, 1, 0], ['_y', '_v', '_u']):
    chi2_avant[plane_num] = []
    chi2_depois[plane_num] = []
    for par0_edges in zip(parameters_bin_edges[0][:-1], parameters_bin_edges[0][1:]):
        chi2_avant[plane_num].append([])
        chi2_depois[plane_num].append([])
        for par1_edges in zip(parameters_bin_edges[1][:-1], parameters_bin_edges[1][1:]):
            this_parameters = [par+plane for par in parameters]
            def sel_function(array):
                return (array['first_last_hit_mask'+plane]) &\
                       (array['llr_012_n'] < -0.1) &\
                        ((array[this_parameters[0]]) > par0_edges[0]) &\
                        ((array[this_parameters[0]]) <= par0_edges[1]) &\
                        ((array[this_parameters[1]]) > par1_edges[0]) &\
                        ((array[this_parameters[1]]) <= par1_edges[1])
            title = title_base + '\nplane {}, {:.2f} < {} < {:.2f} & {:.2f} < {} < {:.2f}'.format(plane_num, par0_edges[0], 
                                                         parameters_legend_names[0],
                                                         par0_edges[1],
                                                         par1_edges[0], 
                                                         parameters_legend_names[1],
                                                         par1_edges[1],)
            
            save_label = '{:04.0f}_{}_{:04.0f}_{:04.1f}_{}_{:04.1f}'.format( 
                                                         par0_edges[0], 
                                                         parameters[0],
                                                         par0_edges[1],
                                                         par1_edges[0], 
                                                         parameters[1],
                                                         par1_edges[1]).replace('.', '').replace('$', '').replace('\\', '')
            
            out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Hit dQ/dx [ADC/cm]', 
                                       title="Before re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.pdf')
            plt.close()
            
            chi2_avant[plane_num][-1].append(out[1]/out[2])
            
            out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Hit dQ/dx [ADC/cm]', 
                                       title="Before re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False,
                                       legend=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.pdf')
            plt.close()
            
            
            
            out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                       title="After re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.pdf')
            plt.close()
            
            chi2_depois[plane_num][-1].append(out2[1]/out2[2])
            
            out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                       title="After re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False,
                                       legend=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.pdf')
            plt.close()

In [ ]:
# summary plots of the chi2

this_folder = 'low_rr_before_after'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/protons/$this_folder/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/protons/{}/'.format(this_folder)

parameters = ['rr', 'pitch']
parameters_legend_names = ['residual range', 'pitch']
parameters_bin_edges = [[0, 2, 5, 10, 20, 30, 50, 150, 300], [0.3, 0.4, 1, 30]]

this_title_base = '\nTrk-score > 0.5, track PID < -0.1\ntrk-vtx distance < 5 cm, start-end fiducial\nplane {}\nMicroBooNE Preliminary'

plt.rcParams.update({'font.size': 10})

def chi2_plot(table, title, save_name):
    plt.pcolormesh(table)
    plt.title(title, loc='left')
    plt.colorbar()
    plt.xticks(range(len(parameters_bin_edges[1])), ['{:.2g}'.format(p) for p in parameters_bin_edges[1]])
    plt.yticks(range(len(parameters_bin_edges[0])), ['{:.2g}'.format(p) for p in parameters_bin_edges[0]])
    plt.xlabel("pitch [cm]")
    plt.ylabel("residual range [cm]")
    
    for i in range(len(parameters_bin_edges[1])-1):
        for j in range(len(parameters_bin_edges[0])-1):
            text = plt.text(i+0.5, j+0.5, "{:.3g}".format(table[j, i]),
                           ha="center", va="center", color="k")
    plt.tight_layout()
    plt.savefig(save_name + '.png', dpi=300)
    plt.savefig(save_name + '.pdf')
    plt.close()

for plane_num in range(3):
    chi2_av = np.array(chi2_avant[plane_num])
    chi2_dep = np.array(chi2_depois[plane_num])
    
    chi2_plot(chi2_av, title=r"$X^2$" + " Before re-calibration" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_avant'.format(plane_num))
    
    chi2_plot(chi2_dep, title=r"$X^2$" + " After re-calibration" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_depois'.format(plane_num))
    
    chi2_plot(chi2_dep-chi2_av, title=r"$X^2_{after} - X^2_{before}/ndof$" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_delta_chi2'.format(plane_num))

    chi2_plot(chi2_dep/chi2_av, r"$X^2_{after}/X^2_{before}$" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_ratio'.format(plane_num))

# LLR PID in function of the phase space

## LLRPID in function of pitch and phi wrt collection plane

In [ ]:
this_folder = 'llr_pid'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/protons/$this_folder/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/protons/{}/'.format(this_folder)

parameters = ['trk_pitch_y', 'trk_abs_theta_yx_y']
parameters_legend_names = ['track pitch Y', r'track $\phi$']
parameters_bin_edges = [np.array([0.3, 0.4, 0.7, 1., 1.5, 30]), np.linspace(0, np.pi/2, 7)]
var = 'llr_012_n'

binning = [25, -1, 1]
title_base = 'Trk-score > 0.5\ntrk-vtx distance < 5 cm, start-end fiducial'

for par0_edges in zip(parameters_bin_edges[0][:-1], parameters_bin_edges[0][1:]):
    for par1_edges in zip(parameters_bin_edges[1][:-1], parameters_bin_edges[1][1:]):
        def sel_function(array):
            return ((array[parameters[0]]) > par0_edges[0]) &\
                    ((array[parameters[0]]) <= par0_edges[1]) &\
                    ((array[parameters[1]]) > par1_edges[0]) &\
                    ((array[parameters[1]]) <= par1_edges[1])
        title = title_base + '\n{:.2f} < {} < {:.2f} & {:.2f} < {} < {:.2f}'.format(par0_edges[0], 
                                                     parameters_legend_names[0],
                                                     par0_edges[1],
                                                     par1_edges[0], 
                                                     parameters_legend_names[1],
                                                     par1_edges[1],)

        save_label = '{:04.1f}_{}_{:04.1f}_{:04.2f}_{}_{:04.2f}'.format( 
                                                     par0_edges[0], 
                                                     parameters[0],
                                                     par0_edges[1],
                                                     par1_edges[0], 
                                                     parameters[1],
                                                     par1_edges[1]).replace('.', '').replace('$', '').replace('\\', '')

        out = my_plotter.plot_pot_normalised_var(variable=var,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel='Track PID - 3 planes',
                                       title=title,
                                       log=False,
                                       subtracted=False,
                                       area_norm=False,
                                       onebin=False)
        plt.savefig(this_save_folder + save_label + '_pot.png', dpi=300)
        plt.savefig(this_save_folder + save_label + '_pot.pdf')
        plt.close()


        out = my_plotter.plot_pot_normalised_var(variable=var,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel='Track PID - 3 planes',
                                       title=title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False)
        plt.savefig(this_save_folder + save_label + '_area_norm.png', dpi=300)
        plt.savefig(this_save_folder + save_label + '_area_norm.pdf')
        plt.close()

        out2 = my_plotter.plot_pot_normalised_var(variable=var,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel='Track PID - 3 planes',
                                       title=title,
                                       log=False,
                                       subtracted=False,
                                       area_norm=False,
                                       legend=False)
        plt.savefig(this_save_folder + save_label + '_pot_noleg.png', dpi=300)
        plt.savefig(this_save_folder + save_label + '_pot_noleg.pdf')
        plt.close()


        out2 = my_plotter.plot_pot_normalised_var(variable=var,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel='Track PID - 3 planes',
                                       title=title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       legend=False)
        plt.savefig(this_save_folder + save_label + '_area_norm_noleg.png', dpi=300)
        plt.savefig(this_save_folder + save_label + '_area_norm_noleg.pdf')
        plt.close()

In [ ]:
## Find event run subrun of events 1mu1p

In [72]:
import pandas as pd
ar = arrays['beam_on']
ar['n_hits_u'] = ar['dqdx_u'].count()
ar['n_hits_v'] = ar['dqdx_v'].count()
ar['n_hits_y'] = ar['dqdx_y'].count()

In [73]:
ar2 = {k: ar[k] for k in ar.keys() & {'evt', 'run', 'sub', 'llr_012_n', 'trk_len', 'trk_score', 'trk_theta', 'trk_phi', 'n_hits_u', 'n_hits_v', 'n_hits_y'}}
df = pd.DataFrame(ar2)
df.set_index(keys=['run', 'sub', 'evt'], inplace=True)

In [74]:
df

n_hits_y  trk_score   trk_phi  n_hits_u     trk_len  n_hits_v  \
run  sub evt                                                                  
5937 11  553       223   0.999962 -2.599607       281  132.534363        62   
         553        89   0.952842  2.883582        18   29.095739        55   
         553         0   0.290909  0.591841        37   16.489212        32   
         553         7   0.868922 -1.261683        11    2.909848         3   
         553         0   0.784093  2.275754         6    1.826744         4   
...                ...        ...       ...       ...         ...       ...   
5189 17  885      1548   1.000000 -1.913854      1139  550.225830       355   
     18  907       144   0.999988  1.549883       352  176.389465       429   
         907         8   0.319347  1.587370        27   12.087006        34   
         914       142   0.983608  2.427637        28   99.485657       125   
         926       343   0.998081 -0.958239       262  129.145493        63   

              trk_theta  llr_012_n  
run  sub evt                        
5937 11  553   0.933568   0.946846  
         553   0.485448  -0.854763  
         553   2.129589  -0.170730  
         553   0.719319  -0.166562  
         553   2.632694  -0.037644  
...                 ...        ...  
5189 17  885   0.278301   0.971057  
     18  907   1.306751   0.918495  
         907   1.494979   0.581683  
         914   0.714128   0.851218  
         926   0.577242   0.938125  

[7342 rows x 8 columns]

In [75]:
df['n_tracks'] = df.groupby(['run', 'sub', 'evt'], sort=False).size()
df['min_llr'] = df.groupby(['run', 'sub', 'evt'], sort=False)['llr_012_n'].min()
df['max_llr'] = df.groupby(['run', 'sub', 'evt'], sort=False)['llr_012_n'].max()
df['all_small_tracks'] = df.groupby(['run', 'sub', 'evt'], sort=False)['trk_len'].max() < 60

In [76]:
df.query('n_tracks == 2 & min_llr < -0.5 & max_llr > 0.5 & all_small_tracks')

n_hits_y  trk_score   trk_phi  n_hits_u    trk_len  n_hits_v  \
run  sub evt                                                                  
5771 21  1089       146   0.889972 -1.942181       140  51.957195        16   
         1089         6   0.902863  1.685074        45  21.361115        61   
     55  2758        65   0.899479 -0.387160        52  30.949495        20   
         2758         7   0.965503  2.242431        23   9.716102        15   
5452 20  1030        47   0.943725 -1.640385       110  52.111237       102   
         1030       109   0.961389  0.181955        44  43.019588        81   
5208 105 5278       116   0.983070 -2.565105        99  44.359993        20   
         5278        59   0.927085  0.845591        26  31.443134        72   
5181 7   352        116   0.979268 -1.339693       165  54.877266        62   
         352         96   0.928592  1.612994        49  58.827267       179   
5385 174 8742       159   0.978482  2.585131        51  52.087036       112   
         8742        35   0.952252  1.097988        87  34.084152        78   
5617 49  2455        14   0.010028 -1.212661        54  26.937893        81   
         2455        33   0.625101 -2.947291        28  13.044603        16   
5526 10  538        171   0.944437 -1.797196        80  54.142921        48   
         538         22   0.977504 -1.925209        26  23.587399        73   
5340 29  1491       112   0.969481 -2.093766       115  55.038532        48   
         1491        67   0.299790  1.534640       106  40.180855        34   
5896 10  529          0   0.999996 -1.625832       139  45.764496        47   
         529         70   0.885911 -1.482963        95  30.008898        26   
5767 108 5420       102   0.591717  1.441063         9  42.186672         0   
         5420        37   0.928837 -1.948469         0  31.206602        50   
5375 21  1084        42   0.786071 -0.214469        41  31.240164        28   
         1084        22   0.120702  1.824601         0  15.826580        40   
5581 8   415         96   0.877951 -0.739018       123  51.734165        38   
         415         89   0.959192  2.653547        19  31.188265        68   
     13  670         53   0.948817  2.145372        42  38.558826       110   
         670         40   0.947788 -0.816859        74  27.082405        20   
     45  2282        72   0.951923  1.055405        16  23.295950        57   
         2282        73   0.958608 -2.167605        19  30.470247        40   
5929 4   217         34   0.908123  1.852084        17  15.810637        46   
         217         26   0.947747 -2.423887        11  16.337719        37   
5419 84  4224       185   0.537545  2.960544         0  58.837902       136   
         4224        45   0.947350 -0.637374         0  16.128788        10   
5914 49  2491        49   0.789153 -0.581818        15  19.914188        37   
         2491        31   0.853164 -0.853670        52  22.499453        22   
5598 72  3614        61   0.948347 -1.585662       119  46.999374        96   
         3614        12   0.952835 -0.744033        20   7.125994         9   
5154 31  1598       132   0.963889  1.879335        44  59.934025       186   
         1598        93   0.969821 -1.083050       111  41.262566        35   
5368 91  4551       186   0.973716 -2.891700       109  58.984619        81   
         4551        92   0.991754 -2.439341         0  30.759878        35   
     92  4647        75   0.968939 -0.708978        53  45.711521        89   
         4647        56   0.949382  1.816683        13  17.859957        45   
5229 8   443        136   0.795208 -0.866052        29  53.229492        38   
         443         26   0.908630  2.116769         0  39.703579       105   
     47  2396        94   0.947021  2.555896        78  38.764175        82   
         2396        32   0.694108 -1.520108        16  14.452335        45   
5906 60  3038        96   0.959404  2.767418        12  33.835373